# Jupyter Interactivity in a Few Cells
## Import and load data

In [ ]:
import pandas as pd
import numpy as np

# Point to the CSV (adjust the path if needed)
csv_path = "bio_toy_DE_results.csv"
df = pd.read_csv(csv_path)
df.head()


Let's use **itables** to interact with the dataframe with sortable headers, search box, pagination

In [ ]:
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)

show(df)

### Filtering
We are using **ipywidgets** + **itables.show** to allow interactive filering using sliders and drop-down menus.

In [ ]:
import ipywidgets as W
from IPython.display import display, clear_output
from itables import show

pathway_dd = W.Dropdown(options=["All"] + sorted(df["pathway"].unique()), value="All", description="Pathway:")
padj_max   = W.FloatLogSlider(value=0.05, base=10, min=-6, max=0, step=0.1, description="padj ≤")
l2fc_min   = W.FloatSlider(value=1.0, min=0.0, max=3.0, step=0.1, description="|log2FC| ≥")

out = W.Output()

def refresh(*_):
    with out:
        clear_output(wait=True)
        filt = df[(df["padj"] <= padj_max.value) & (df["log2FoldChange"].abs() >= l2fc_min.value)]
        if pathway_dd.value != "All":
            filt = filt[filt["pathway"] == pathway_dd.value]
        show(filt.sort_values(["padj", "log2FoldChange"]))

for w in (pathway_dd, padj_max, l2fc_min):
    w.observe(refresh, names="value")

refresh()
display(W.VBox([W.HBox([pathway_dd, padj_max, l2fc_min]), out]))


## Interactive Volcano Plot

For live-updating plots and pan/zoom. The first line of the cell tells the notebook to render the **matplotlib** plot as an interactive widget.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipympl

fig, ax = plt.subplots()
sc = ax.scatter(df["log2FoldChange"], -np.log10(df["padj"]+1e-300), s=12, alpha=0.6)
ax.set_xlabel("log2 fold change")
ax.set_ylabel("-log10(padj)")
ax.set_title("Volcano (interactive thresholds)")

# Threshold lines (updated by sliders)
vline = ax.axvline(1.0, linestyle="--", linewidth=1)
vline2 = ax.axvline(-1.0, linestyle="--", linewidth=1)
hline = ax.axhline(-np.log10(0.05), linestyle="--", linewidth=1)

def update(l2fc=1.0, padj=0.05):
    # Update lines
    vline.set_xdata([l2fc, l2fc])
    vline2.set_xdata([-l2fc, -l2fc])
    hline.set_ydata([-np.log10(padj)])
    # Highlight significant points by resizing markers
    sig = (df["padj"] <= padj) & (df["log2FoldChange"].abs() >= l2fc)
    sizes = np.where(sig, 30, 12)
    sc.set_sizes(sizes)
    fig.canvas.draw_idle()

W.interact(update,
           l2fc=W.FloatSlider(min=0.0, max=3.0, step=0.1, value=1.0, description="|log2FC|"),
           padj=W.FloatLogSlider(base=10, min=-6, max=0, step=0.1, value=0.05, description="padj"));


## Histogram with Interactive Bin Count

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

fig2, ax2 = plt.subplots()

def plot_sig_summary(padj=0.05, l2fc=1.0):
    ax2.clear()
    sig = df[(df["padj"] <= padj) & (df["log2FoldChange"].abs() >= l2fc)]
    counts = sig["pathway"].value_counts().sort_values(ascending=False)
    counts.plot(kind="bar", ax=ax2)
    ax2.set_ylabel("# genes (significant)")
    ax2.set_title(f"Significant genes by pathway (padj≤{padj:g}, |log2FC|≥{l2fc:g})")
    fig2.canvas.draw_idle()

W.interact(plot_sig_summary,
           padj=W.FloatLogSlider(base=10, min=-6, max=0, step=0.1, value=0.05, description="padj"),
           l2fc=W.FloatSlider(min=0.0, max=3.0, step=0.1, value=1.0, description="|log2FC|"));
